In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from ark import io
from pathlib import Path
import spatialdata as sd
from spatialdata.models import C
from spatial_image import SpatialImage

/Users/srivarra/.pyenv/versions/3.11.4/envs/ark-spatial/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/srivarra/.pyenv/versions/3.11.4/envs/ark-spatial/lib/python3.11/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()
/Users/srivarra/.pyenv/versions/3.11.4/envs/ark-spatial/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55948 instead
  warnings.warn(
/Users/srivarra/.pyenv/versions/3.11.4/envs/ark-spatial/lib/python3.11/site-packages/dask/config.py:694: UserWar

## 1. Convert a FOV cohort to a spatial data cohort

Set the path of the cohort

In [ ]:
cohort_dir = Path("../data/example_dataset/image_data")

Load the cohort

In [ ]:
sdata: sd.SpatialData = io.load_cohort(cohort_dir)

View the cohort

In [ ]:
sdata

We can access any Image in the Spatial Data object by using the image name as a key in the `images` dictionary.

In [ ]:
sdata.images.keys()

In [ ]:
sdata.images["fov0"]

Even better, we can access a coordinate system. Each FOV is mapped to a coordinate system which is a unique identifier for a collection of Spatial Data objects. We will see the benefits of this later on.

In [ ]:
sdata.coordinate_systems

In [ ]:
sdata.images["fov0"]

We can look at the channels in our cohort

In [ ]:
sdata.images["fov0"].c

Select a few channels from `fov0`

In [ ]:
sdata.images["fov0"].sel({C: ["CD3", "CD4", "CD8"]})

We can select FOVs of interest as well

In [ ]:
sdata.sel(elements=["fov0", "fov8"])

We can broadcast a query across axes for certain coordinates too.

In [ ]:
sdata.query.bounding_box(
    axes=["x", "y"],
    min_coordinate=[0, 0],
    max_coordinate=[256, 256],
    target_coordinate_system="fov0",
)

Lets save the Spatial Data object as a OME ZARR Store

In [3]:
cohort_sd_save_path = Path("../data/cohorts/example_cohort.ome.zarr")

In [ ]:
sdata.write(file_path=cohort_sd_save_path)

We can load the Zarr store to a spatial data object as well.

In [4]:
sdata = sd.read_zarr(store=cohort_sd_save_path)

In [5]:
type(sdata._get_group_for_element("fov0", element_type="images").store)

zarr.storage.FSStore

In [6]:
sdata.coordinate_systems

['fov1',
 'fov4',
 'fov3',
 'fov6',
 'fov5',
 'fov2',
 'fov7',
 'global',
 'fov8',
 'fov0',
 'fov10',
 'fov9']

In [8]:
import xbatcher as xb
from xbatcher import BatchAccessor

In [19]:
f = xb.BatchGenerator(
    ds=sdata.sel(elements=["fov0"]).images["fov0"],
    input_dims={"x": 2048, "y": 2048},
)
# sdata.sel(elements=["fov0"])
len(f)

ValueError: input sample length must be less than or equal to the dimension length, but the sample length of 2048 is greater than the dimension length of 512 for x

In [58]:
from spatialdata.models import X, Y

fs = []

for f in xb.BatchGenerator(
    ds=sdata.sel(elements=["fov0"]).images["fov0"],
    input_dims={"x": 128, "y": 128},
):
    fs.append(f)
    print(type(f))

<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>
<class 'spatial_image.SpatialImage'>


In [65]:
import flox.xarray as fx
from spatialdata.models import X, Y

In [77]:
nucs = ["H3K9ac", "H3K27me3"]
mems = ["CD14", "CD45", "ECAD"]

[['H3K9ac', 'H3K27me3'], ['CD14', 'CD45', 'ECAD']]


In [111]:
import numpy as np

# TODO post issue about how to group the nucs and the mems and sum each of them up.
fx.xarray_reduce(
    sdata.sel(elements=["fov0"]).images["fov0"],
    # by=[nucs, mems],
    nucs,
    mems,
    dim={C: [nucs, mems]},
    func="sum",
    engine="flox",
    expected_groups=[nucs, mems],
)

ValueError: When grouping by multiple variables, expected_groups must be a tuple of either arrays or objects convertible to an array (like lists). For example `expected_groups=(np.array([1, 2, 3]), ['a', 'b', 'c'])`.Received a list instead. When grouping by a single variable, you can pass an array or something convertible to an array for convenience: `expected_groups=['a', 'b', 'c']`.

In [54]:
import xarray as xr

In [57]:
xr.combine_by_coords(data_objects=fs) == sdata.sel(elements=["fov0"]).images["fov0"]

<xarray.Dataset>
Dimensions:  (c: 22, y: 512, x: 512)
Coordinates:
  * c        (c) <U11 'CD3' 'CD4' 'CD8' 'CD14' ... 'Ki67' 'PD1' 'SMA' 'Vim'
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 507.5 508.5 509.5 510.5 511.5
Data variables:
    fov0     (c, y, x) bool dask.array<chunksize=(1, 512, 512), meta=np.ndarray>

In [59]:
list(
    xb.BatchGenerator(
        ds=sdata.sel(elements=["fov0"]).images["fov0"],
        input_dims={"x": 128, "y": 128},
    )
)

[<xarray.SpatialImage 'fov0' (c: 22, y: 128, x: 128)>
 array([[[7.98785128e-03, 9.64157935e-03, 8.01919308e-03, ...,
          1.42103789e-04, 1.13440549e-03, 2.12501059e-03],
         [4.76125535e-03, 5.86257409e-03, 7.14094052e-03, ...,
          7.86317105e-04, 1.06452452e-03, 1.97150209e-03],
         [1.82170223e-03, 3.33288428e-03, 6.43185293e-03, ...,
          2.63729575e-03, 2.36747065e-03, 3.38933081e-03],
         ...,
         [9.71920555e-04, 6.19818573e-04, 9.88918706e-04, ...,
          0.00000000e+00, 7.22415105e-04, 0.00000000e+00],
         [4.27915336e-04, 2.09446211e-04, 1.25346987e-05, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.02745823e-03, 3.83607781e-04, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 
        [[9.75698233e-03, 1.20721338e-02, 1.15622357e-02, ...,
          6.22877292e-03, 7.92760774e-03, 1.30784884e-02],
         [6.43406995e-03, 8.82666558e-03, 1.10815288e-02, ...,
          9.